## Исследование надёжности заёмщиков


[Введение](#beginning)

1. [Изучение общей информации](#start)

2. [Предобработка данных](#preprocessing)

    2.1. [Обработка пропусков и замена типа данных](#null)
    
    2.2. [Обработка данных в столбце Трудовой стаж_начало (замена неправильных данных на нули)](#employed)
    
    2.3. [Обработка данных в столбце Возраст клиента](#dob)
    
    2.4. [Обработка данных в стобце Стаж_продолжение (замена пропусков на средние)](#employed_2)
    
    2.5. [Обработка данных в столбце Тип занятости](#income_type)
    
    2.6. [Обработка данных в столбце Доход](#income)
    
    2.7. [Обработка данных в столбце Количество детей](#children)
    
3. [Обработка дубликатов](#duplicates)

    3.1. [Обработка дубликатов в столбце Образование](#education)
    
    3.2. [Обработка дубликатов в столбце семейное положение](#family)
    
    3.3. [Обработка дубликатов в столбце Пол](#gender)
    
4. [Лемматизация](#lemmatiz)

5. [Категоризация данных](#categorization)

    5.1. [Категоризация по размеру дохода](#income_cat)
    я по количеству детей](#children_cat)
    
6. [Ответы на вопросы](#questions)

    5.2. [Категоризаци
7. [Общий вывод](#summary)
    

<a id="beginning"></a>
### Введение

В ходе данной работы будет проанализировано, влияет ли семейное положение, количество детей, уровень дохода и цель кредита на его возврат в срок.
Рассмотрим исходную таблицу, удалим пропуски, дубликаты, скорректируем некорректные значение, произведём категоризацию полей для анализа.

<a id="start"></a>
### 1. Изучение общей информации 

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
m = Mystem() 
from collections import Counter
data = pd.read_csv('/datasets/data.csv')


In [2]:

data.to_csv('название_датиасета.расширение', index=False)
print(data.info())

data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [4]:
data.sample(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14945,2,-3270.716057,31,среднее,1,женат / замужем,0,F,сотрудник,0,109122.931329,покупка жилья для сдачи
18292,0,-2243.865825,50,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,148317.102299,покупка недвижимости
2178,0,370309.695325,56,среднее,1,женат / замужем,0,F,пенсионер,0,114487.376518,жилье
16647,0,-277.421453,32,среднее,1,женат / замужем,0,F,сотрудник,0,136820.774559,строительство собственной недвижимости
12287,1,336482.924112,66,среднее,1,вдовец / вдова,2,F,пенсионер,1,154095.489940,покупка коммерческой недвижимости
8427,1,-2910.519543,41,высшее,0,женат / замужем,0,F,компаньон,0,383807.533908,покупка жилья для сдачи
4223,2,-2194.470501,35,среднее,1,женат / замужем,0,F,сотрудник,0,233809.547291,операции с жильем
18790,0,-3316.534503,46,СРЕДНЕЕ,1,женат / замужем,0,F,сотрудник,0,130476.597013,получение высшего образования
17509,20,-461.939717,31,среднее,1,Не женат / не замужем,4,F,сотрудник,0,90937.893204,жилье
5623,0,NaN,30,высшее,0,гражданский брак,1,F,компаньон,0,NaN,на проведение свадьбы


### Вывод

С первого взгляда на таблицу можно заметить следующие моменты, с которыми необходимо будет поработать:
* Трудовой стаж: много отрицательных значений, значения не целые, много пропусков, некоторые значения некорректные, очень большие;
* Образование: часть данных написана capslock;
* Доход: не целые значения, для удобства обработки необходимо будет переводить в целочисленный формат, много пропусков;
* Цель получения кредита: смысловые дубликаты (напр., покупка жилья и покупка жилья для семьи).

<a id="preprocessing"></a>
### 2. Предобработка данных

<a id="null"></a>
#### 2.1. Обработка пропусков и замена типа данных

Произведём подготовительные операции для предобработки данных:
* Выведем уникальные данные по столбцам

In [5]:
print('Названия столбцов')
data.columns

Названия столбцов


Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [6]:
unique_education = data['education'].unique()
print('Уникальные данные в столбце Образование')
unique_education

Уникальные данные в столбце Образование


array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [7]:
unique_children = data['children'].unique()
print('Уникальные данные в столбце Количество детей')
unique_children

Уникальные данные в столбце Количество детей


array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [8]:
unique_education_id = data['education_id'].unique()
print('Уникальные данные в столбце Идентификатор образования')
unique_education_id

Уникальные данные в столбце Идентификатор образования


array([0, 1, 2, 3, 4])

In [9]:
unique_family_status = data['family_status'].unique()
print('Уникальные данные в столбце Семейное положение')
unique_family_status

Уникальные данные в столбце Семейное положение


array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [10]:
unique_family_status_id = data['family_status_id'].unique()
print('Уникальные данные в столбце Идентификатор семейного положения')
unique_family_status_id

Уникальные данные в столбце Идентификатор семейного положения


array([0, 1, 2, 3, 4])

In [11]:
unique_gender = data['gender'].unique()
print('Уникальные данные в столбце Пол')
unique_gender

Уникальные данные в столбце Пол


array(['F', 'M', 'XNA'], dtype=object)

In [12]:
unique_income_type = data['income_type'].unique()
print('Уникальные данные в столбце Тип занятости')
unique_income_type

Уникальные данные в столбце Тип занятости


array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [13]:
unique_debt = data['debt'].unique()
print('Уникальные данные в столбце Задолженность')
unique_debt

Уникальные данные в столбце Задолженность


array([0, 1])

In [14]:
unique_purpose = data['purpose'].unique()
print('Уникальные данные в столбце Цель получения кредита')
unique_purpose

Уникальные данные в столбце Цель получения кредита


array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

* Определим количество пропущенных значений

In [15]:
print(data.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


* Проверим гипотезу о том, что пропуски в стоблцах days_employed и total_income совпадают

In [16]:
data[(data['days_employed'].isnull()) & (data['total_income'].isnull())]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Гипотеза подтвердилась: действительно, у всех безработных не указан доход.

<a id="employed"></a>
#### 2.2. Обработка данных в столбце Трудовой стаж_начало (замена неправильных данных на нули)

Далее предполагаем, что отрицательные значения в столбце days_employed возникли из-за дефисов в начале данных, т.к. сами отрицательные данные похожи на правду, а вот с положительными придётся разобраться.
Сначала сделаем все данные в этом столбце положительными. И переведём в года, чтобы было ясно, какие значения некорректны.

In [17]:
new_names = ['children', 'years_employed', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']
data.set_axis(new_names, axis = 'columns', inplace = True)
print(data.columns)
data['years_employed'] = abs(data['years_employed'])
data['years_employed'] = data['years_employed']/365
data.sort_values(by = 'years_employed', ascending = False).head(10)

Index(['children', 'years_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')


,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,1100.699727,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,1100.591265,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,1100.479708,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,1100.477991,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,1100.448904,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
4697,0,1100.369953,56,среднее,1,женат / замужем,0,F,пенсионер,0,48242.322502,покупка недвижимости
13420,0,1100.327762,63,Среднее,1,гражданский брак,1,F,пенсионер,0,51449.788325,сыграть свадьбу
17823,0,1100.313632,59,среднее,1,женат / замужем,0,F,пенсионер,0,152769.694536,покупка жилья для сдачи
10991,0,1100.251585,56,среднее,1,в разводе,3,F,пенсионер,0,39513.517543,получение дополнительного образования
8369,0,1100.247814,58,среднее,1,женат / замужем,0,F,пенсионер,0,175306.312902,образование


Посчитаем количество строк, где стаж больше возраста.

In [18]:
data [data['years_employed'] > data['dob_years']]['years_employed'].count()

3519

Корректного стажа более 50 лет в таблице нет. Стаж менее 1 также некорректен

In [19]:
data [data['years_employed'] > 50]['years_employed'].count()

3446

In [20]:
data [data['years_employed'] < 1]['years_employed'].count()

1827

Напишем функцию для замены некорректных значений стажа на 0, чтобы потом обработать эти строки вместе с теми, которые содержат значения NaN

In [21]:
def employed(years):
    
        if years > 50:
                return 0
        else:
            if years < 1:
                return 0
            else:
                return years


Проверим корректность работы функции.

In [22]:
print(employed(51)) 
print(employed(49))
print(employed(0.1)) 

0
49
0


Применим функцию к нашим данным:

In [23]:
data['employed'] = data['years_employed'].apply(employed)
data.head(5) 

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677
4,0,932.235814,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,0.000000


Заменим столбец стажа на новосформированный

In [24]:
data['years_employed'] = data['employed']
data.head(5) 

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677
4,0,0.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,0.000000


Заменим пропущенные значения NaN на нули, используя метод fillna, чтобы потом обработать все нули (и от этой замены, и от предыдущей) вместе

In [25]:
data['years_employed'] = data['years_employed'].fillna(0)
print(data.isnull().sum())

children               0
years_employed         0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
employed            2174
dtype: int64


Проверим себя, подсчитав общее количество нулевых значений в столбце years_employed с учётом обоих замен.

In [26]:
data [data['years_employed'] == 0]['years_employed'].count()

7447

Необходимо будет заменить слишком большие значения стажа и нулевые значения стажа на средние по каждому возрасту.
Но для этого необходимо разобраться со столбцом dob_years.



<a id="dob"></a>
#### 2.3. Обработка данных в столбце Возраст клиента

Посмотрим, сколько строк попало в каждый возраст

In [27]:
print(data['dob_years'].value_counts())

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


Заменим нулевые значения на средние

In [28]:
dobyears_avg = data['dob_years'].mean()
dobyears_avg
data['dob_years'] = data['dob_years'].replace(0, dobyears_avg)
print(data['dob_years'].value_counts())

35.00000    617
40.00000    609
41.00000    607
34.00000    603
38.00000    598
42.00000    597
33.00000    581
39.00000    573
31.00000    560
36.00000    555
44.00000    547
29.00000    545
30.00000    540
48.00000    538
37.00000    537
50.00000    514
43.00000    513
32.00000    510
49.00000    508
28.00000    503
45.00000    497
27.00000    493
56.00000    487
52.00000    484
47.00000    480
54.00000    479
46.00000    475
58.00000    461
57.00000    460
53.00000    459
51.00000    448
59.00000    444
55.00000    443
26.00000    408
60.00000    377
25.00000    357
61.00000    355
62.00000    352
63.00000    269
64.00000    265
24.00000    264
23.00000    254
65.00000    194
66.00000    183
22.00000    183
67.00000    167
21.00000    111
43.29338    101
68.00000     99
69.00000     85
70.00000     65
71.00000     58
20.00000     51
72.00000     33
19.00000     14
73.00000      8
74.00000      6
75.00000      1
Name: dob_years, dtype: int64


В результате замены на среднее значение получили данные типа float64. Заменим их обратно на целочисленные.

In [29]:
data['dob_years'] = data['dob_years'].astype('int')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
years_employed      21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
employed            19351 non-null float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.1+ MB
None


In [30]:
data['dob_years'] = data['dob_years'].astype('uint8')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
years_employed      21525 non-null float64
dob_years           21525 non-null uint8
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
employed            19351 non-null float64
dtypes: float64(3), int64(4), object(5), uint8(1)
memory usage: 2.0+ MB
None


Напишем функцию для распределения клиентов по возрастным категориям. Её в дальнейшем будем использовать для заполнения некорректных значений в столбце Стаж. Т.к. клиентов младше 18 лет быть не может, то у функции будет такая логика:
Клиенты до 44 лет  — категория «молодой возраст»;
Клиенты от 45 до 59 лет  — «средний возраст»;
Клиенты старше 60 лет - «пенсионеры».

In [31]:
def age_group(age):
    
        if age < 45:
                return 'молодой возраст'
        if age <= 59:
                return 'средний возраст'
        return 'пенсионеры' 

Протестриуем работу функции для каждого возраста

In [32]:
print(age_group(44)) 
print(age_group(45)) 
print(age_group(59)) 
print(age_group(60)) 

молодой возраст
средний возраст
средний возраст
пенсионеры


Применим функцию к нашим данным и создадим отдельный столбец с распределением по возрастным категориям.

In [33]:
data['age_group'] = data['dob_years'].apply(age_group)
data.head(5) 

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed,age_group
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912,молодой возраст
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860,молодой возраст
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637,молодой возраст
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677,молодой возраст
4,0,0.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,0.000000,средний возраст


<a id="employed_2"></a>
####  2.4. Обработка данных в стобце Стаж_продолжение (замена пропусков на средние)

Теперь можно обработать пропущенные значения в столбце Стаж в соответствии с возрастной категорией.
Объявим переменную для массива данных без нулей по данному столбцу, чтобы посчитать по ним средние.

In [34]:
data_without_null = data [data['years_employed'] != 0]
data_without_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14078 entries, 0 to 21524
Data columns (total 14 columns):
children            14078 non-null int64
years_employed      14078 non-null float64
dob_years           14078 non-null uint8
education           14078 non-null object
education_id        14078 non-null int64
family_status       14078 non-null object
family_status_id    14078 non-null int64
gender              14078 non-null object
income_type         14078 non-null object
debt                14078 non-null int64
total_income        14078 non-null float64
purpose             14078 non-null object
employed            14078 non-null float64
age_group           14078 non-null object
dtypes: float64(3), int64(4), object(6), uint8(1)
memory usage: 1.5+ MB


In [35]:
data_without_null.head(5)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed,age_group
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912,молодой возраст
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860,молодой возраст
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637,молодой возраст
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677,молодой возраст
5,0,2.537495,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,2.537495,молодой возраст


Определим средний стаж по возрастным группам, на который будем заменять нули в столбце со стажем

In [36]:
print(data_without_null['years_employed'].mean())
years_employed_mean=data_without_null.groupby('age_group')['years_employed'].mean()
print(years_employed_mean)
years_employed_mean_young = years_employed_mean[0]
print(years_employed_mean_young)
years_employed_mean_old = years_employed_mean[1]
print(years_employed_mean_old)
years_employed_mean_middle = years_employed_mean[2]
print(years_employed_mean_middle)

7.201441527771776
age_group
молодой возраст     5.994822
пенсионеры         11.170514
средний возраст     9.324281
Name: years_employed, dtype: float64
5.994821582165485
11.170513986143673
9.324281371054607


Создадим функцию для замены нулей в столбце стажа на средние значения по возрастным группам и применим её к data

In [37]:
def years_employed_correction(row):
    years_employed = row['years_employed']
    age_group = row['age_group']
      
    if years_employed == 0:
        if age_group == 'молодой возраст':
            return years_employed_mean_young
        else:
            if age_group == 'пенсионеры':
                return years_employed_mean_old
            else:
                return years_employed_mean_middle
    else:
        return years_employed
    


<div class="alert alert-block alert-success">
<b>Success:</b> Отличная функция!

Проверим, как работает функция

In [38]:
row_values = [0, 'молодой возраст']
row_columns = ['years_employed', 'age_group']
row = pd.Series(data=row_values, index=row_columns)
print(years_employed_correction(row))

row_values = [0, 'пенсионеры']
row_columns = ['years_employed', 'age_group']
row = pd.Series(data=row_values, index=row_columns)
print(years_employed_correction(row))

row_values = [0, 'средний возраст']
row_columns = ['years_employed', 'age_group']
row = pd.Series(data=row_values, index=row_columns)
print(years_employed_correction(row))

5.994821582165485
11.170513986143673
9.324281371054607


И применим её к data

In [39]:
data['years_employed_new'] = data.apply(years_employed_correction, axis=1)
data.head(5)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed,age_group,years_employed_new
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912,молодой возраст,23.116912
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860,молодой возраст,11.026860
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637,молодой возраст,15.406637
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677,молодой возраст,11.300677
4,0,0.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,0.000000,средний возраст,9.324281


Переведём значения стажа в целочисленный формат:

In [40]:
data['years_employed_new'] = data['years_employed_new'].astype('int')
data.head(5)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed,age_group,years_employed_new
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912,молодой возраст,23
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860,молодой возраст,11
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637,молодой возраст,15
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677,молодой возраст,11
4,0,0.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,0.000000,средний возраст,9


In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 15 columns):
children              21525 non-null int64
years_employed        21525 non-null float64
dob_years             21525 non-null uint8
education             21525 non-null object
education_id          21525 non-null int64
family_status         21525 non-null object
family_status_id      21525 non-null int64
gender                21525 non-null object
income_type           21525 non-null object
debt                  21525 non-null int64
total_income          19351 non-null float64
purpose               21525 non-null object
employed              19351 non-null float64
age_group             21525 non-null object
years_employed_new    21525 non-null int64
dtypes: float64(3), int64(5), object(6), uint8(1)
memory usage: 2.3+ MB


In [42]:
data['years_employed_new'] = data['years_employed_new'].astype('uint8')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 15 columns):
children              21525 non-null int64
years_employed        21525 non-null float64
dob_years             21525 non-null uint8
education             21525 non-null object
education_id          21525 non-null int64
family_status         21525 non-null object
family_status_id      21525 non-null int64
gender                21525 non-null object
income_type           21525 non-null object
debt                  21525 non-null int64
total_income          19351 non-null float64
purpose               21525 non-null object
employed              19351 non-null float64
age_group             21525 non-null object
years_employed_new    21525 non-null uint8
dtypes: float64(3), int64(4), object(6), uint8(2)
memory usage: 2.2+ MB


Применим к столбцу со стажем формат uint8, т.к. все значения целые до 255.
Экономия места за счёт использования типа данных uint8 - 4,35%

Проверим, не осталось ли значений менее 1 (т.к. мы корректировали не только нули, но и значения менее 1)

In [43]:
data['years_employed_new'].min()

1

Заменим прежний столбец со Стажем на новосформированный

In [44]:
data['years_employed'] = data['years_employed_new']
data.head(5)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed,age_group,years_employed_new
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912,молодой возраст,23
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860,молодой возраст,11
2,0,15,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637,молодой возраст,15
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677,молодой возраст,11
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,0.000000,средний возраст,9


И удалим ненужные столбцы из таблицы

In [45]:
del data['employed']
del data['years_employed_new']
data.head(5)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,молодой возраст
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,молодой возраст
2,0,15,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,молодой возраст
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,молодой возраст
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,средний возраст


Проверим информацию о таблице для проверки применённой обработки

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
years_employed      21525 non-null uint8
dob_years           21525 non-null uint8
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: float64(1), int64(4), object(6), uint8(2)
memory usage: 1.8+ MB


<a id="income_type"></a>
#### 2.5. Обработка данных в столбце Тип занятости

В столбце тип занятости нет дубликатов и пропусков, однако в группе "предприниматель", "безработный", "студент" и "в декрете" малочисленны, сравнивать их с другими, более многочисленными, некорректно, поэтому объединим их по смыслу, возрасту и уровню дохода с бОльшими категориями.

In [47]:
print(data['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64


In [48]:
data['income_type'] = data['income_type'].replace('предприниматель', 'компаньон')
data['income_type'] = data['income_type'].replace('студент', 'сотрудник')
data['income_type'] = data['income_type'].replace('безработный', 'сотрудник')
data['income_type'] = data['income_type'].replace('в декрете', 'сотрудник')

In [49]:
print(data['income_type'].value_counts())

сотрудник      11123
компаньон       5087
пенсионер       3856
госслужащий     1459
Name: income_type, dtype: int64


<a id="income"></a>
#### 2.6. Обработка данных в столбце Доход

Остались пропуски в столбце Доход.
Выведем статистическую информацию по столбцу:

In [50]:
print('Минимальное значение в столбце доход')
print(data['total_income'].min())
print('Максимальное значение в столбце доход')
print(data['total_income'].max())
print('Среднее значение по столбцу доход')
print(data['total_income'].mean())
print('Медиана по столбцу доход')
total_income_median = data['total_income'].median()
print(total_income_median)

Минимальное значение в столбце доход
20667.26379327158
Максимальное значение в столбце доход
2265604.028722744
Среднее значение по столбцу доход
167422.30220817294
Медиана по столбцу доход
145017.93753253992


Т.к. в столбце Доход нет нереальных значений, требующих корректировки, но разброс значений велик, для замены пропущенных значений будем использовать медиану по каждому типу занятости.

In [51]:
print(data['income_type'].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий']


In [52]:
data_without_null_2 = data [data['total_income'] != 0]
data_without_null_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
years_employed      21525 non-null uint8
dob_years           21525 non-null uint8
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: float64(1), int64(4), object(6), uint8(2)
memory usage: 2.0+ MB


In [53]:

income_type_median=data_without_null_2.groupby('income_type')['total_income'].median()
print(income_type_median)

income_type_median_gossluzh = income_type_median[0]
print(income_type_median_gossluzh)

income_type_median_companion = income_type_median[1]
print(income_type_median_companion)

income_type_median_pensioneer = income_type_median[2]
print(income_type_median_pensioneer)

income_type_median_worker = income_type_median[3]
print(income_type_median_worker)


income_type
госслужащий    150447.935283
компаньон      172396.000846
пенсионер      118514.486412
сотрудник      142563.419337
Name: total_income, dtype: float64
150447.9352830068
172396.00084601453
118514.48641164352
142563.41933652188


Напишем функцию для возвращения в новый столбец с доходом медиану по каждому типу занятости при условии, если значение дохода не заполнено, проверим её корректность и применим к нашему массиву данных, предварительно заменив пропущенные значения NaN методом fillna на нули.

In [54]:
def income_correction(row):
    total_income = row['total_income']
    income_type = row['income_type']
      
    if total_income == 0:
        if income_type == 'госслужащий':
            return income_type_median_gossluzh
        else:
            if income_type == 'компаньон':
                return income_type_median_companion
            else:
                if income_type == 'пенсионер':
                    return income_type_median_pensioneer
                else:
                    if income_type == 'сотрудник':
                        return income_type_median_worker
                    
    else:
        return total_income
       

In [55]:
row_values = [0, 'сотрудник']
row_columns = ['total_income', 'income_type']
row = pd.Series(data=row_values, index=row_columns)
print(income_correction(row))

row_values = [1000000, 'сотрудник']
row_columns = ['total_income', 'income_type']
row = pd.Series(data=row_values, index=row_columns)
print(income_correction(row))

row_values = [0, 'компаньон']
row_columns = ['total_income', 'income_type']
row = pd.Series(data=row_values, index=row_columns)
print(income_correction(row))

142563.41933652188
1000000
172396.00084601453


In [56]:
data['total_income'] = data['total_income'].fillna(value=0)
data.head(5)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,молодой возраст
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,молодой возраст
2,0,15,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,молодой возраст
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,молодой возраст
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,средний возраст


In [57]:
data['total_income_new'] = data.apply(income_correction, axis=1)
data.head(15)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_new
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,молодой возраст,253875.639453
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,молодой возраст,112080.014102
2,0,15,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,молодой возраст,145885.952297
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,молодой возраст,267628.550329
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,средний возраст,158616.077870
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,молодой возраст,255763.565419
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,молодой возраст,240525.971920
7,0,9,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,средний возраст,135823.934197
8,2,18,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,молодой возраст,95856.832424
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,молодой возраст,144425.938277


Переведём получившиеся значения дохода в целочисленный формат и заменим столбец с доходом на новый.
А потом удалим ненужный столбец.

In [58]:
data['total_income_new'] = data['total_income_new'].astype('int')
data.head(5)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_new
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,молодой возраст,253875
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,молодой возраст,112080
2,0,15,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,молодой возраст,145885
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,молодой возраст,267628
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,средний возраст,158616


In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
children            21525 non-null int64
years_employed      21525 non-null uint8
dob_years           21525 non-null uint8
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
age_group           21525 non-null object
total_income_new    21525 non-null int64
dtypes: float64(1), int64(5), object(6), uint8(2)
memory usage: 2.0+ MB


In [60]:
data['total_income_new'] = data['total_income_new'].astype('uint32')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
children            21525 non-null int64
years_employed      21525 non-null uint8
dob_years           21525 non-null uint8
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
age_group           21525 non-null object
total_income_new    21525 non-null uint32
dtypes: float64(1), int64(4), object(6), uint32(1), uint8(2)
memory usage: 1.9+ MB


Применим к столбцу Доход формат uint32, т.к. все значения целые до 4294967295 .
Экономия места за счёт использования типа данных uint32 - 5%.
Общая экономия места за счёт использования подходящих типов данных - около 10%

In [61]:
data['total_income'] = data['total_income_new']

In [62]:
del data['total_income_new']
data.head(5)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,молодой возраст
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,молодой возраст
2,0,15,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,молодой возраст
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,молодой возраст
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,средний возраст


In [63]:
print(data['total_income'].mean())

165208.96422764228


Проверим, что пропусков в таблице больше не осталось:

In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
years_employed      21525 non-null uint8
dob_years           21525 non-null uint8
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null uint32
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: int64(4), object(6), uint32(1), uint8(2)
memory usage: 1.8+ MB


<a id="children"></a>
#### 2.7. Обработка данных в столбце Количество детей

Проанализируем данные в столбце Количество детей. Используем метод value_counts для подсчёта значений по категориям.

In [65]:
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Данные, содержащие -1, скорее всего, вызваны неправильным заполнением анкеты через дефис. Применим к ним функцию взятия по модулю. А строки с некорректными данными, содержащие 20 детей, можно просто удалить, т.к. они составляют всего 0,3% от общей выборки.

In [66]:
data['children'] = abs(data['children'])
print(data['children'].value_counts())

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


In [67]:
data_new = data [data['children'] != 20]
print(data_new['children'].value_counts())
data = data_new
data.info()

0    14149
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21449 entries, 0 to 21524
Data columns (total 13 columns):
children            21449 non-null int64
years_employed      21449 non-null uint8
dob_years           21449 non-null uint8
education           21449 non-null object
education_id        21449 non-null int64
family_status       21449 non-null object
family_status_id    21449 non-null int64
gender              21449 non-null object
income_type         21449 non-null object
debt                21449 non-null int64
total_income        21449 non-null uint32
purpose             21449 non-null object
age_group           21449 non-null object
dtypes: int64(4), object(6), uint32(1), uint8(2)
memory usage: 1.9+ MB


### Вывод

1. Мы избавились от неопределённых значений NaN  в таблице, которые, скорее всего, возникли по причине нежелания клиентов указывать эти данные. Или по причине того, что данные, предоставленные банку по другой форме (например, 3-НДФЛ вместо 2-НДФЛ) неправильно занеслись в базу. Необходимо обратиться с данной проблемой к руководству.

2. Осуществили замена пропущенных значений на средние для каждой возрастной категории в столбце Стаж и на медианы по каждому типу занятости в столбце Доход.

3. Заменили нулевые значения возраста на средние.

4. Объединили категории типа занятости, чтобы не оставлять категорий с единичными значениями.

5. Удалили строки с некорректным количеством детей и исправили ошибки ввода.

4. Заменили числовые данные на целочисленные.

### Замена типа данных

Замена типа данных на целочисленный выполнена к столбцам Стаж и Доход в предыдущих разделах после обработки пропусков и некорректных значений.

<a id="duplicates"></a>
### 3. Обработка дубликатов

<a id="education"></a>
#### 3.1. Обработка дубликатов в столбце Образование

Переведём все значения в нижний регистр

In [68]:
data['education_lowercase'] = data['education'].str.lower()
data['education'] = data['education_lowercase']
del data['education_lowercase']
data.head(5)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,молодой возраст
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,молодой возраст
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,молодой возраст
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,молодой возраст
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,средний возраст


И определим уникальные значение в столбце методом unique

In [69]:
unique_education = data['education'].unique()
print('Уникальные данные в столбце Образование')
print(unique_education)

Уникальные данные в столбце Образование
['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


После приведения значений в столбце Образование к нижнему регистру дубликатов в столбце Образование не осталось.

<a id="family"></a>
#### 3.2.  Обработка дубликатов в столбце семейное положение

В столбце Семейное положение дубликатов нет, что было определено методом unique при анализе общей информации о данных.

<a id="gender"></a>
#### 3.3. Обработка дубликатов и данных в столбце Пол

В столбце Пол дубликатов также нет (также определили при анализе общей информации о таблице).

Посмотрим, сколько раз встречается непонятное значение XNA в столбце Пол.

In [70]:
print(data['gender'].value_counts())

F      14189
M       7259
XNA        1
Name: gender, dtype: int64


Значение XNA встречается только 1 раз.
Т.к. нам не требуется находить зависимость возврата кредита от пола, строку с данным значением можно оставить и не корректировать.


### Вывод

Все дубликаты в столбце Образование были полными. И исправились за счёт изменения регистра.
Далее будет рассмотрены дубликаты в столбце Цель получения кредита

<a id="lemmatiz"></a>
### 4. Лемматизация

Т.к. столбцец  Цель получения кредита содержит смысловые дубликаты (напр., покупка жилья и покупка жилья для семьи), лемматизируем его. Создадим словарь лемм для выделения основных целей получения кредита.

In [71]:
from pymystem3 import Mystem
m = Mystem() 
from collections import Counter


def lemmatize_data(row): 
    lemmas = m.lemmatize(row)
    return lemmas
data['purpose_lemm'] = data['purpose'].apply(lemmatize_data)
print('Словарь лемм столбца "purpose": ', Counter(data['purpose_lemm'].sum()))




Словарь лемм столбца "purpose":  Counter({' ': 33553, '\n': 21449, 'недвижимость': 6346, 'покупка': 5893, 'жилье': 4458, 'автомобиль': 4299, 'образование': 4007, 'с': 2911, 'операция': 2600, 'свадьба': 2339, 'свой': 2231, 'на': 2222, 'строительство': 1874, 'высокий': 1369, 'получение': 1313, 'коммерческий': 1308, 'для': 1292, 'жилой': 1230, 'сделка': 940, 'заниматься': 908, 'дополнительный': 905, 'проведение': 773, 'сыграть': 770, 'сдача': 652, 'семья': 640, 'собственный': 629, 'со': 629, 'ремонт': 610, 'подержанный': 485, 'подержать': 473, 'приобретение': 462, 'профильный': 434})


In [72]:
data.head(5)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_lemm
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,молодой возраст,"[покупка, , жилье, \n]"
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,молодой возраст,"[приобретение, , автомобиль, \n]"
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,молодой возраст,"[покупка, , жилье, \n]"
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,молодой возраст,"[дополнительный, , образование, \n]"
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,средний возраст,"[сыграть, , свадьба, \n]"


Выделим следующие категории целей, под которые берут кредит:
* недвижимость (леммы: недвижимость, жилье)
* образование (леммы: образование)
* автомобиль (леммы: автомобиль)
* свадьба (леммы: свадьба)
* ремонт (леммы: ремонт)

Напишем функцию для категоризации данных по видам целей, под которые взят кредит, проверим её работу и применим к нашим данным.

In [73]:
def purpose_typing(row):
    if 'ремонт' in row:
        return 'ремонт'
    if 'недвижимость' in row or 'жилье' in row:
        return 'недвижимость'
    if 'образование' in row:
        return 'образование'
    if 'автомобиль' in row:
        return 'автомобиль'
    if 'свадьба' in row:
        return 'свадьба'
    else:
        return 'не определена'


In [74]:
print(purpose_typing('образование'))
print(purpose_typing('квартира'))
print(purpose_typing('жилье'))
print(purpose_typing('ремонт'))
print(purpose_typing('автомобиль'))

образование
не определена
недвижимость
ремонт
автомобиль


In [75]:
data['purpose_type'] = data['purpose_lemm'].apply(purpose_typing)
data.head(5)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_lemm,purpose_type
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,молодой возраст,"[покупка, , жилье, \n]",недвижимость
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,молодой возраст,"[приобретение, , автомобиль, \n]",автомобиль
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,молодой возраст,"[покупка, , жилье, \n]",недвижимость
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,молодой возраст,"[дополнительный, , образование, \n]",образование
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,средний возраст,"[сыграть, , свадьба, \n]",свадьба


Посчитаем, под какие цели был взят кредит сколько раз.

In [76]:
print(data['purpose_type'].value_counts())

недвижимость    10194
автомобиль       4299
образование      4007
свадьба          2339
ремонт            610
Name: purpose_type, dtype: int64


### Вывод

В ходе лемматизации были выделены 5 основных целей, под которые клиенты брали кредит.

<a id="categorization"></a>
### 5. Категоризация данных

Уже есть категории по семейному положению. В разделе Лемматизация созданы категории по целям взятия кредита.
Осталось создать категории по размеру дохода и количеству детей.

<a id="income_cat"></a>
#### 5.1. Категоризация по размеру дохода

Определим минимальные и максимальные доходы по каждому типу занятости.

In [77]:
print('Минимальные зарплаты по каждому типу занятости')
data.groupby('income_type')['total_income'].min()

Минимальные зарплаты по каждому типу занятости


income_type
госслужащий    29200
компаньон      28702
пенсионер      20667
сотрудник      21367
Name: total_income, dtype: uint32

In [78]:
print('Максимальные зарплаты по каждому типу занятости')
data.groupby('income_type')['total_income'].max()

Максимальные зарплаты по каждому типу занятости


income_type
госслужащий     910451
компаньон      2265604
пенсионер       735103
сотрудник      1726276
Name: total_income, dtype: uint32

Создадим функцию для категоризации уровней дохода:
* менее  100000 в месяц - низкие;
* от 100001 до 150000 - средние;
* более 150000 - высокие.
Проверим работоспособность функции и применим к нашим данным.
Данные категории выбраны с учётом количества данных.

In [79]:
def income_typing(income):
    if income <= 100000:
        return 'низкий'
    if income <= 150000:
        return 'средний'
    return 'высокий'

In [80]:
print(income_typing(20000))
print(income_typing(60000))
print(income_typing(20000000))

низкий
низкий
высокий


In [81]:
data['income_group'] = data['total_income'].apply(income_typing)
data.head()

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_lemm,purpose_type,income_group
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,молодой возраст,"[покупка, , жилье, \n]",недвижимость,высокий
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,молодой возраст,"[приобретение, , автомобиль, \n]",автомобиль,средний
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,молодой возраст,"[покупка, , жилье, \n]",недвижимость,средний
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,молодой возраст,"[дополнительный, , образование, \n]",образование,высокий
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,средний возраст,"[сыграть, , свадьба, \n]",свадьба,высокий


In [82]:
data['income_group'].value_counts()

высокий    9806
средний    7191
низкий     4452
Name: income_group, dtype: int64

<a id="children_cat"></a>
#### 5.2. Категоризация по количеству детей
Т.к. группы с числом детей более 3 малочисленны, объединим семьи с количеством детей 2 и более в одну подкатегорию.
Создадим функцию для категоризации по числу детей, проверим её и применим к данным.

In [83]:
print(data['children'].value_counts())

0    14149
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


In [84]:
def children_typing(children):
    if children == 0:
        return 0
    if children == 1:
        return 1
    return 2

In [85]:
print(children_typing(0))
print(children_typing(1))
print(children_typing(2))
print(children_typing(3))

0
1
2
2


In [86]:
data['children_group'] = data['children'].apply(children_typing)
data.head()

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_lemm,purpose_type,income_group,children_group
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,молодой возраст,"[покупка, , жилье, \n]",недвижимость,высокий,1
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,молодой возраст,"[приобретение, , автомобиль, \n]",автомобиль,средний,1
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,молодой возраст,"[покупка, , жилье, \n]",недвижимость,средний,0
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,молодой возраст,"[дополнительный, , образование, \n]",образование,высокий,2
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,средний возраст,"[сыграть, , свадьба, \n]",свадьба,высокий,0


Проверим, корректно ли применилась функция к массиву данных.

In [87]:
print(data['children_group'].value_counts())

0    14149
1     4865
2     2435
Name: children_group, dtype: int64


### Вывод

Были выделены 3 группы для категоризации по количеству детей в семье и 3 для категоризации по уровню дохода.
Группы по целям кредита были выделены в главе Лемматизация.
Группы по семейному положению сохраняем исходные.

<a id="questions"></a>
### 6. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [88]:
data_grouped_children = data.groupby('children_group').agg({'debt': ['count', 'sum']})
data_grouped_children['percent_debt'] = data_grouped_children['debt']['sum'] / data_grouped_children['debt']['count']

print(data_grouped_children)

                 debt       percent_debt
                count   sum             
children_group                          
0               14149  1063     0.075129
1                4865   445     0.091470
2                2435   225     0.092402


In [89]:
import numpy as np
data_pivot_children = data.pivot_table(index=['children_group'], values='debt', aggfunc=[np.sum,'count'])
data_pivot_children['percent_debt'] = data_pivot_children['sum']['debt'] / data_pivot_children['count']['debt']
data_pivot_children_sorted = data_pivot_children.sort_values(by = 'percent_debt', ascending = False)
data_pivot_children_sorted

,sum,count,percent_debt
,debt,debt,
children_group,,,
2,225,2435,0.092402
1,445,4865,0.091470
0,1063,14149,0.075129


### Вывод

Выявлена зависимость между наличием детей и возвратом кредита в срок:
чем больше детей, тем больше вероятность невозврата кредита в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [90]:
data_grouped_family = data.groupby('family_status').agg({'debt': ['count', 'sum']})
data_grouped_family['percent_debt'] = data_grouped_family['debt']['sum'] / data_grouped_family['debt']['count']

print(data_grouped_family)

                        debt      percent_debt
                       count  sum             
family_status                                 
Не женат / не замужем   2804  273     0.097361
в разводе               1193   84     0.070411
вдовец / вдова           956   63     0.065900
гражданский брак        4165  385     0.092437
женат / замужем        12331  928     0.075257


In [91]:
data_pivot_family = data.pivot_table(index=['family_status'], values='debt', aggfunc=[np.sum,'count'])
data_pivot_family['percent_debt'] = data_pivot_family['sum']['debt'] / data_pivot_family['count']['debt']
data_pivot_family_sorted = data_pivot_family.sort_values(by = 'percent_debt', ascending = False)
data_pivot_family_sorted

,sum,count,percent_debt
,debt,debt,
family_status,,,
Не женат / не замужем,273,2804,0.097361
гражданский брак,385,4165,0.092437
женат / замужем,928,12331,0.075257
в разводе,84,1193,0.070411
вдовец / вдова,63,956,0.065900


### Вывод

Также выявлена зависимость между семейным положением и возвратом кредита в срок: люди, не обременённые семейными узами (как просто не женатые, так и в гражданском браке), более склонны к просрочкам кредита.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [92]:
data_grouped_income = data.groupby('income_group').agg({'debt': ['count', 'sum']})
data_grouped_income['percent_debt'] = data_grouped_income['debt']['sum'] / data_grouped_income['debt']['count']

print(data_grouped_income)

              debt      percent_debt
             count  sum             
income_group                        
высокий       9806  759     0.077402
низкий        4452  354     0.079515
средний       7191  620     0.086219


In [93]:
data_pivot_income = data.pivot_table(index=['income_group'], values='debt', aggfunc=[np.sum,'count'])
data_pivot_income['percent_debt'] = data_pivot_income['sum']['debt'] / data_pivot_income['count']['debt']
data_pivot_income_sorted = data_pivot_income.sort_values(by = 'percent_debt', ascending = False)
data_pivot_income_sorted

,sum,count,percent_debt
,debt,debt,
income_group,,,
средний,620,7191,0.086219
низкий,354,4452,0.079515
высокий,759,9806,0.077402


### Вывод

Зависимость между уровнем дохода и возвратом кредита в срок также имеется, но не такая явно выраженная.
Средний класс чаще имеет просрочки по кредитам.
Предполагаем, что дюди с высоким доходом ответственно подходят к уплате кредитов и имеют на это финансовые возможности.
А люди с низким доходом и берут небольшие суммы, и боятся ухудшить кредитную историю, поэтому тоже стараются не допускать просрочек. Но проверить данные предположения по имеющимся данным нет возможности.

- Как разные цели кредита влияют на его возврат в срок?

In [94]:
data_grouped_purpose = data.groupby('purpose_type').agg({'debt': ['count', 'sum']})
data_grouped_purpose['percent_debt'] = data_grouped_purpose['debt']['sum'] / data_grouped_purpose['debt']['count']

print(data_grouped_purpose)

               debt      percent_debt
              count  sum             
purpose_type                         
автомобиль     4299  401     0.093278
недвижимость  10194  745     0.073082
образование    4007  369     0.092089
ремонт          610   35     0.057377
свадьба        2339  183     0.078239


In [95]:
data_pivot_purpose = data.pivot_table(index=['purpose_type'], values='debt', aggfunc=[np.sum,'count'])
data_pivot_purpose['percent_debt'] = data_pivot_purpose['sum']['debt'] / data_pivot_purpose['count']['debt']
data_pivot_purpose_sorted = data_pivot_purpose.sort_values(by = 'percent_debt', ascending = False)
data_pivot_purpose_sorted

,sum,count,percent_debt
,debt,debt,
purpose_type,,,
автомобиль,401,4299,0.093278
образование,369,4007,0.092089
свадьба,183,2339,0.078239
недвижимость,745,10194,0.073082
ремонт,35,610,0.057377


### Вывод

Как и можно было предположить, люди берущие кредит на приобретение автомобиля или образование, более склонны к просрочкам, т.к. решения о данных кредитах не такие продуманные, а также потому, что данной категории заёмщиков требуются денежные средства даже в меньшем размере, чем на приобретение недвижимости.
На удивление, те, кто берёт кредит на организацию свадьбы, также оказались вполне добросовестными заёмщиками. Скорее всего, это можно объяснить тем, что после свадьбы можно погасить кредит за счёт подаренных денег. Но подтведить или опровергнуть данные гипотезы с учётом имеющихся данных не представляется возможным.
Однако самыми добросовестными заёмщиками оказались клиенты, берущие кредит на ремонт жилья. Это можно объяснить устоявшимся бытом этих людей и их зрелостью, т.к. жильё у этих людей уже есть.
Также добросоветстность заёмщиков, берущих кредит на ремонт и на недвижимость может быть объяснена тем, что в залоге у банка при выдачи таких кредитов остаётся недвижимость.
Для проверки данных гипотез необходимы дополнительные данные.

<a id="summary"></a>
### 7. Общий вывод

Была проведена обработка данных для решения поставленной задачи о выявлении зависимостей между наличием детей, семейным положением, уровнем дохода, целями кредита и возвратом кредита в срок.
    
1. В ходе обработки данных было выявлено, что около 10% данных содержали пропуски, скорее всего вызванные добавлением в базу данных возможности принятия справки о доходах не по форме банка и не по 2-НДФЛ.
Были выявлены ошибки ввода: CapsLock, опечатки при цифровом вводе, отрицательные значения. Данные ошибки можно в дальнейшем устранить, создав ограничения при вводе данных в поля.
Необходимо обратиться к руководству банка для решения данных проблем.
    
2. Проанализировав данные, мы можем сказать, что выявлены следующие зависимости:
    
    - Чем больше детей, тем больше вероятность невозврата кредита в срок.
    
    - Люди, не обременённые семейными узами (как просто не женатые, так и в гражданском браке), более склонны к просрочкам кредитам.
    
    - Средний класс чаще имеет просрочки по кредитам.
    
    - Люди берущие кредит на приобретение автомобиля или образование, более склонны к просрочкам.
    
    - Самыми добросовестными заёмщиками оказались клиенты, берущие кредит на ремонт жилья